<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de la red de vuelos con GraphFrames - I <span style="color: green">[solución]</span></div>
<!-- Paulo Villegas, 2021 - v. 1.3 -->

In [ ]:
spark

In [ ]:
sc.setLogLevel('INFO')

# 1. Leer datos

Los datos de aeropuertos y rutas están almacenados en ficheros CSV. Los cargamos en DataFrames

In [ ]:
airports = spark.read.csv('../DATA/openflights/airports.csv.bz2', inferSchema=True, header=True, nullValue='\\N')

In [ ]:
airports.rdd.getNumPartitions()

In [ ]:
airports.write.mode('overwrite').parquet('/vagrant/test2')

In [ ]:
routes = spark.read.csv('../DATA/openflights/routes.csv.bz2', inferSchema=True, header=True,  nullValue='\\N')

Esquema de datos leídos

In [ ]:
airports.printSchema()

In [ ]:
airports.head()

In [ ]:
routes.printSchema()

Tamaño de datos

In [ ]:
routes.limit(5).toPandas()

In [ ]:
airports.filter(airports.IATA == 'ATL').toPandas()

In [ ]:
routes.filter(routes.SourceAirport == 'ATL').count()

In [ ]:
airports.count(), routes.count()

# 2. Construcción del grafo

In [ ]:
import graphframes as gf

## 2.1 Adaptar nodos y enlaces

In [ ]:
vertices = airports.withColumnRenamed('AirportId', 'id')

In [ ]:
edges_full = routes.withColumnRenamed('SourceAirportId', 'src').withColumnRenamed('DestAirportId', 'dst')

In [ ]:
edges_full.count()

## 2.2 Filtrado de enlaces espúreos

Eliminamos las rutas cuyo aeropuerto de entrada o de salida no está en la lista de nodos

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">El resultado de esta operación es correcto, pero no se ha implementado de la forma adecuada.<br>
    Ejercicio: averigüe por qué y reescribala usando una forma más apropiada</div

<div class="alert alert-block alert-info">

### 2.2.1 versión usando un set local de Python
Esto es  una traslación directa de lo realizado con NetworkX -- como tal no sería válido si el grafo es grande, ya que en la operación descargamos totalmente la lista de vérticas a local (al hacer `collect`), lo que podría saturar la memoria de nuestro proceso local. Además obligamos luego a Spark a subir ese set de vértices desde local a todos los ejecutores.
    
Es decir, funciona, pero estamos haciendo trampa.
</div>

In [ ]:
airport_list = vertices.rdd.map(lambda r: r.id).collect()

In [ ]:
airport_list = set(airport_list)

In [ ]:
edges = edges_full.filter( edges_full['src'].isin(airport_list) & edges_full['dst'].isin(airport_list) )

In [ ]:
edges.count()

<div class="alert alert-block alert-info">
    
### 2.2.2 version realizada íntegramente en Spark
Esta operación se puede también realizar íntegramente en Spark (sin necesidad de traer la lista de aeropuertos a local) usando _join_. Esta es la forma más correcta de hacerlo.
</div>

In [ ]:
edges_full.printSchema()

In [ ]:
# Escogemos del DataFrame de aeropuertos sólo la columna 'id' para hacer el join
airport_id = vertices.select('id')

In [ ]:
# Primera restricción: condición sobre el aeropuerto de salida
edges = edges_full.join( airport_id, edges_full.src == airport_id.id ).drop(airport_id.id)

In [ ]:
# Segunda restricción: condición sobre el aeropuerto de entrada
edges = edges.join( airport_id, edges.dst == airport_id.id ).drop(airport_id.id)

In [ ]:
edges.count()

## 2.3 Creación del grafo

In [ ]:
g = gf.GraphFrame(vertices, edges)

# 3. Grabar a disco un grafo

In [ ]:
g.vertices.write.mode('overwrite').parquet('OUT/flights-vertices')
g.edges.write.mode('overwrite').parquet('OUT/flights-edges')

# 4. Operaciones

## 4.1 Cálculo de grados

In [ ]:
airport_names = vertices.select('id', 'Name', 'City', 'IATA')

In [ ]:
g.degrees.printSchema()

In [ ]:
grados = g.degrees

In [ ]:
grados.groupBy('degree').count().orderBy('count', ascending=False).toPandas()

In [ ]:
grados.orderBy('degree', ascending=False ).limit(10).toPandas()

In [ ]:
ordered_degrees = grados.orderBy('degree', ascending=False ).join(airport_names, 'id')

In [ ]:
ordered_degrees.limit(10).toPandas()

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Nota: estos valores son mucho más altos que los obtenidos en NetworkX sobre la misma red.<br>Ejercicio: averiguar por qué y proponer una solución.</div>

<div class="alert alert-block alert-info">

### 4.1.1 Eliminar duplicados en enlaces

</div>

La razón de fondo de esta diferencia de comportamiento es que nuestro grafo en realidad es un **Multigrafo**: entre un par de nodos cualquiera puede haber (y de hecho hay) _más de un enlace_ : cada conexión entre dos aeropuertos es un enlace, y una conexión puede ser servida por un número variables de vuelos, de compañías aéreas distintas.

Por ejemplo, veamos las conexiones entre Valladolid y Barcelona:

In [ ]:
g.edges.filter((g.edges.SourceAirport == 'VLL') & (g.edges.DestAirport == 'BCN')).toPandas()

Como vemos, hay 3 vuelos en el grafo, que se convierten en 3 enlaces. NetworkX lo tiene en cuenta a la hora de calcular el grado de un nodo, pero GraphFrames simplemente cuenta las filas, y por eso acaba con un valor muy superior al grafo base

Podemos eliminar fácilmente esos enlaces múltiples usando el método `dropDuplicates` de un DataFrame de Spark, elminando las filas que tengan el mismo `src` y mismo `dst` (nos quedamos con solo uno de los vuelos):

In [ ]:
edges2 = edges.dropDuplicates(['src', 'dst'])
g2 = gf.GraphFrame(vertices, edges2)

In [ ]:
g2.degrees.join(airport_names, 'id').orderBy('degree', ascending=False ).limit(10).toPandas()

Hemos reducido bastante la diferencia, pero seguimos obteniendo valores mucho mayores. De hecho, son *aproximadamente el doble*, y eso nos da una pista ...

<div class="alert alert-block alert-info">

### 4.1.2 Eliminar vuelos de vuelta

</div>

Hay que tener en cuenta que aunque no sea ya un multigrafo, esto es un grafo _dirigido_ , y es perfectamente posible que entre dos nodos haya 2 enlaces, uno en cada sentido. De hecho sería lo más probable: cada vuelo suele tener un vuelo de vuelta, por el modelo de trabajo de las líneas aéreas. Por ejemplo, volviendo a Valladolid-Barcelona

In [ ]:
g2.edges.filter((g.edges.SourceAirport.isin(['VLL', 'BCN'])) & 
                (g.edges.DestAirport.isin(['VLL','BCN']))).toPandas()

Como vemos, efectivamente hay dos vuelos entre los dos nodos, uno en cada sentido, y de nuevo GraphFrames simplemente cuenta filas, así que nos da un grado mayor del real

Para solucionarlo podemos pensar en calcular solo uno de los grados (grado de entrada o grado de salida):

In [ ]:
g2.outDegrees.join(airport_names, 'id').orderBy('outDegree', ascending=False ).limit(10).toPandas()

In [ ]:
g2.inDegrees.join(airport_names, 'id').orderBy('inDegree', ascending=False ).limit(10).toPandas()

Estos números son ya razonables, y dan una idea mucho más real del grado "crudo" de conectividad entre nodos. Aunque vemos que todavía hay alguna diferencia: el grado de entrada no es igual al grado de salida en muchos casos (aunque la diferencia es pequeña). Esto puede deberse:
* a que efectivamente hay algún vuelo asimétrico entre dos aeropuertos (quizás porque la compañía aérea triangula el avión con un tercer aeropuerto)
* a que faltan datos de algún vuelo en nuestro dataset (recordemos que no es un dataset oficial, sino un conjunto open source recolectado por [OpenFlights](https://openflights.org/data.html))

<div class="alert alert-block alert-info">

### 4.1.3 Consolidar en enlaces únicos

</div>

Si queremos asegurarnos de efectivamente podar el grafo a solo un enlace entre cada par de aeropuertos, elminando la duplicidad ida-vuelta y vuelta-ida, podemos también hacerlo manipulando el DataFrame de enlaces. Aunque en este caso vamos a necesitar algo más de trabajo:

In [ ]:
import pyspark.sql.functions as F

edges3 = edges.withColumn('edge_idx', F.array_join(F.array_sort(F.array(F.col("src"),
                                                                        F.col("dst"))), "-" ))

Es decir:
* nos creamos una nueva columna `edge_idx`
* esa columna la construimos como una concatenación del aeropuerto de origen y de destino
* pero antes de concatenar **ordenamos** la pareja de aeropuertos, para que en `edge_idx` aparezca el mismo valor para las filas `apto1-apto2` y `apto2-apto1`

In [ ]:
edges3.limit(3).toPandas()

Ahora, si eliminamos duplicados basándonos en la columna `edge_idx`, estaremos seguros de tener *solo una* fila para cada pareja de aeropuertos

In [ ]:
g3 = gf.GraphFrame(vertices, edges3.dropDuplicates(['edge_idx']))

Podemos entonces usar el grado normal de los nodos, y GraphFrame nos dará ahora el valor que buscamos:

In [ ]:
g3.degrees.join(airport_names, 'id').orderBy('degree', ascending=False ).limit(10).toPandas()

In [ ]:
# Grabamos a disco el dataset corregido
g3.vertices.write.mode('overwrite').parquet('../OUT/flights-trim-vertices')
g3.edges.write.mode('overwrite').parquet('../OUT/flights-trim-edges')

## 4.2 Cálculo de componentes conectados

In [ ]:
%%time

sc.setCheckpointDir('tmp/cc')
cc = g.connectedComponents()

In [ ]:
cc.count()

In [ ]:
cc.printSchema()

Calculemos el tamaño de cada componente
<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Ejercicio: escriba las líneas necesarias</div>

<div class="alert alert-block alert-info">
    Lo solucionamos con un <code>groupby</code> más una agregación. Por último ordenamos el resultado.
</div>

In [ ]:
import pyspark.sql.functions as F

In [ ]:
cc_size = cc.groupby('component').agg(F.count('component').alias('num'))

In [ ]:
cc_size = cc_size.sort('num', ascending=False)

In [ ]:
cc_size.toPandas()

Veamos el contenido de alguna de las comunidades

In [ ]:
c2 = cc.filter(cc.component == '5642').sort('Country', 'City')

In [ ]:
c2.toPandas()

In [ ]:
airports.filter(airports.Country == 'Namibia').toPandas()

## 4.3 Búsqueda de caminos

In [ ]:
ruta = g.bfs("IATA = 'MAD'", "IATA = 'JFK'")

In [ ]:
ruta.toPandas()

In [ ]:
ruta = g.bfs("IATA = 'MAD'", "Country = 'Mongolia'")

In [ ]:
ruta.toPandas()

In [ ]:
ruta = g.bfs("City = 'Vigo'", "City = 'Mendoza'")

In [ ]:
ruta.toPandas()

In [ ]:
vertices.filter(vertices.id == 2650).toPandas()

## 4.4 Búsqueda de patrones

El esquema típico es:
1. Selección de subgrafos que siguen un determinado patrón
2. Filtrado de los subgrafos por propiedades de los nodos o los enlaces

Comenzamos con una búsqueda: seleccionamos todos los subgrafos de 3 vuelos consecutivos que se pueden formar. Al hacerlo Spark dará usará los nombres de nodos y enlaces en los patrones de búsqueda para dar nombre a las columnas del dataframe resultado.

In [ ]:
motif1 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")

In [ ]:
motif1.printSchema()

In [ ]:
motif1.count()

Quedémonos con las que hagan los 3 vuelos con la misma compañía


In [ ]:
motif2 = motif1.filter( (motif1['e1'].Airline == motif1['e2'].Airline) & 
                        (motif1.e2.Airline == motif1.e3.Airline) )

In [ ]:
motif2.count()

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Ejercicio: añada ahora una condición adicional: los tres aeropuertos deben estar en países distintos</div>

<div class="alert alert-block alert-info">
Aplicamos un filtro adicional sobre los resultados anteriores, forzando todas las condiciones sobre los nodos. Como son 4 aeropuertos en total, tenemos que asegurarnos de que ningún par de aeropuertos está en el mismo país; eso son C(4,2) = 6 condiciones en total.
</div>

In [ ]:
motif3 = motif2.filter( (motif2.a.Country != motif2.b.Country) & (motif2.b.Country != motif2.c.Country) & 
                        (motif2.c.Country != motif2.d.Country) & (motif2.c.Country != motif2.a.Country) & 
                        (motif2.d.Country != motif2.b.Country) & (motif2.d.Country != motif2.a.Country) )

In [ ]:
motif3.count()

In [ ]:
from IPython.display import display
import pandas as pd

d = motif3.limit(10).toPandas()
with pd.option_context('display.max_colwidth', 100):
    display(d)